In [10]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get('https://www.dcinside.com')

driver.find_element(By.CSS_SELECTOR,'#preSWord').click() #검색창 누르기
search = driver.find_element(By.CSS_SELECTOR, '#preSWord')
search.send_keys("공강 때")
# search.send_keys(Keys.RETURN)
time.sleep(2)
driver.find_element(By.CSS_SELECTOR,'#searchSubmit').click()  #검색버튼 누르기

driver.find_element(By.XPATH, '//*[@id="container"]/div/section[2]/div[1]/div/div/div/button[2]').click() #정확도 순으로 바꾸기

In [12]:
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 맨 아래로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 페이지 로드 대기
    time.sleep(2)

    # 새로운 스크롤 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 스크롤이 더 이상 안 내려가면 종료
    if new_height == last_height:
        break
    
    last_height = new_height

In [13]:
driver.find_element(By.XPATH, '//*[@id="container"]/div/section[2]/div[1]/div/a').click() #게시글 더보기

In [14]:
#pageNum = 2
post_links = []

for pageNum in range(15,20):
    # 디시인사이드 검색 결과 페이지에 접속
    driver.get(f'https://search.dcinside.com/post/p/{pageNum}/sort/accuracy/q/.EA.B3.B5.EA.B0.95.20.EB.95.8C')
    driver.implicitly_wait(10)  # 페이지 로딩 대기
    soup = bs(driver.page_source, 'lxml')
    
    # 각 게시글의 링크를 추출
    titles = soup.select('a.tit_txt')
    
    # 링크가 없으면 더 이상 페이지가 없다고 간주하고 종료
    if len(titles) == 0:
        break

    # 링크를 리스트에 추가
    for title in titles:
        url = title['href']
        post_links.append(url)
    

# # 추출된 링크들을 확인
print(f"총 {len(post_links)}개의 링크를 수집했습니다.")

총 125개의 링크를 수집했습니다.


In [15]:
import time
import random  # random 모듈 임포트

# 각 게시글의 정보를 담을 리스트
post_details_list = []

def scrape_post(link):
    driver.get(link)
    
    # 페이지 로딩 대기 (3초에서 6초 사이)
    time.sleep(random.uniform(3, 6))

    soup = bs(driver.page_source, 'lxml')

    # 제목 추출
    title = soup.select_one('.title_subject').text.strip()

    # 내용 추출
    content = soup.select_one('.write_div').text.strip()

    # 댓글 추출
    comments = [comment.text.strip() for comment in soup.select('.usertxt.ub-word')]

    # URL과 날짜 추출
    url = link
    date = soup.select_one('.gall_date').text.strip()  # 게시글 날짜 추출

    return {
        'title': title,
        'content': content,
        'comments': comments,
        'url': url,
        'date': date
    }

# 각 링크에 대해 게시글 정보 추출 및 리스트에 추가
for link in post_links:
    try:
        post_details = scrape_post(link)
        post_details_list.append(post_details)
        
        # 각 게시글 크롤링 후 추가 대기 (2초에서 5초 사이)
        time.sleep(random.uniform(2, 5))
        
    except Exception as e:
        print(f"Error occurred with link {link}: {str(e)}")
        
        # 오류 발생 시 더 길게 대기 (5초에서 10초 사이)
        time.sleep(random.uniform(5, 10))

# 추출된 게시글 정보 출력
for post in post_details_list:
    print(f"제목: {post['title']}")
    print(f"내용: {post['content']}")
    print(f"댓글: {post['comments']}")
    print(f"URL: {post['url']}")
    print(f"날짜: {post['date']}")
    print("-" * 50)

# 브라우저 종료
driver.quit()

Error occurred with link https://gall.dcinside.com/board/view/?id=lovelive_new1&no=126362: 'NoneType' object has no attribute 'text'
제목: 지역감정 떡밥 586 같아서 끼기 싫은데 나도 당한적 있긴함ㅋㅋ
내용: 약간 서먹하긴 해도 공강 때마다 밥 먹자고 먼저 다가와서  점심팸에 끼워주고 친구로 지내던 놈 있었는데  뒤에서 내 뒷담 까다 걸리고 손절함 그새끼 목포 출신이었음
댓글: []
URL: https://gall.dcinside.com/mgallery/board/view/?id=chzzk&no=2004460
날짜: 2024.05.03 17:36:05
--------------------------------------------------
제목: 3만 필드만 와도 좋겠다
내용: 가끔 공강때 돌렸는데 3/128 이면 슬퍼...
댓글: []
URL: https://gall.dcinside.com/mgallery/board/view/?id=baf2042&no=261973
날짜: 2024.06.08 03:09:18
--------------------------------------------------
제목: 공강
내용: 공강때 생활관 복귀 가능한가요- dc official App
댓글: ['작년부터 침대 사용 가능 ㅇㅇ. 물론 교장님 지시라 바뀔 가능성은 有', '관습기간때는 모르겠다~']
URL: https://gall.dcinside.com/mgallery/board/view/?id=academy&no=25632
날짜: 2023.02.15 23:57:27
--------------------------------------------------
제목: 공강 때 병원이나 갈까
내용: 
댓글: ['갔다왔으']
URL: https://gall.dcinside.com/mini/board/view/?id=bornin04&no=235619
날짜: 2023.03.2

In [17]:
data = []

# post_details_list에서 데이터 추출 및 댓글을 분리하여 추가
for post in post_details_list:
    title = post['title']
    content = post['content']
    url = post['url']
    date = post['date']
    
    # post['comments']는 이미 리스트이므로, 각 댓글을 개별 행으로 추가
    for comment in post['comments']:
        data.append({
            '제목': title,
            '내용': content,
            '댓글': comment,
            'URL': url,
            '날짜': date
        })

# 데이터 프레임 생성
df = pd.DataFrame(data)

# 데이터프레임 확인
df

,제목,내용,댓글,URL,날짜
0,공강,공강때 생활관 복귀 가능한가요- dc official App,작년부터 침대 사용 가능 ㅇㅇ. 물론 교장님 지시라 바뀔 가능성은 有,https://gall.dcinside.com/mgallery/board/view/...,2023.02.15 23:57:27
1,공강,공강때 생활관 복귀 가능한가요- dc official App,관습기간때는 모르겠다~,https://gall.dcinside.com/mgallery/board/view/...,2023.02.15 23:57:27
2,공강 때 병원이나 갈까,,갔다왔으,https://gall.dcinside.com/mini/board/view/?id=...,2023.03.22 11:13:22
3,화수공강때 할 거 추천 좀,미시 예습이나 돌릴까,일루와,https://gall.dcinside.com/mgallery/board/view/...,2022.09.04 22:57:33
4,화수공강때 할 거 추천 좀,미시 예습이나 돌릴까,관정 6층서 영화보기 거기서 군대가기 전에 심슨 시즌 1부터 정주행함ㅋㅋㅋ,https://gall.dcinside.com/mgallery/board/view/...,2022.09.04 22:57:33
...,...,...,...,...,...
269,공강일때,너네뭐함- dc official App,흥,https://gall.dcinside.com/mini/board/view/?id=...,2023.03.05 23:43:45
270,공강일때,너네뭐함- dc official App,애들이랑 과방가서 선배랑 얘기,https://gall.dcinside.com/mini/board/view/?id=...,2023.03.05 23:43:45
271,시간표 짤때 무조건 공강하루는 필수엿.,5일은 힘들어요ㅇㅇ,통학이 왕복 4시간이라..,https://gall.dcinside.com/board/view/?id=tr&no...,2023.10.23 12:18:20
272,시간표 짤때 무조건 공강하루는 필수엿.,5일은 힘들어요ㅇㅇ,난 4일도 자살말리던데,https://gall.dcinside.com/board/view/?id=tr&no...,2023.10.23 12:18:20


In [18]:
df.to_csv('디시인사이드_검색어_크롤링_공강때(15-19페이지).csv',index=False, encoding='utf-8-sig')